# 01 - Build Final Dataset

Merge all raw CSV files into one master dataset

In [ ]:
import pandas as pd
import numpy as np
import os
import sys

sys.path.append(os.path.abspath('../src'))
from src.data_pipeline import load_raw_data, merge_datasets, handle_missing_values, build_final_dataset

## Load All Raw Data

In [8]:
# Load all raw files using the pipeline function
try:
    dfs = load_raw_data('../data/raw')
    print(f"[OK] Total files loaded: {len(dfs)}")
except Exception as e:
    print(f"[ERROR] Failed to load raw data: {e}")
    raise

📂 Loading raw CSV files...

  ✓ sales                (743268, 7)     ['Unnamed: 0', 'date', 'item_id', 'quantity', 'price_base', 'sum_total', 'store_id']
  ✓ price_history        (69863, 6)      ['Unnamed: 0', 'date', 'item_id', 'price', 'code', 'store_id']
  ✓ discounts_history    (374674, 9)     ['Unnamed: 0', 'date', 'item_id', 'sale_price_before_promo', 'sale_price_time_promo', 'promo_type_code', 'doc_id', 'number_disc_day', 'store_id']
  ✓ catalog              (21981, 9)      ['Unnamed: 0', 'item_id', 'dept_name', 'class_name', 'subclass_name', 'item_type', 'weight_volume', 'weight_netto', 'fatness']
  ✓ stores               (4, 5)          ['store_id', 'division', 'format', 'city', 'area']
  ✓ online               (112341, 7)     ['Unnamed: 0', 'date', 'item_id', 'quantity', 'price_base', 'sum_total', 'store_id']
  ✓ markdowns            (898, 7)        ['Unnamed: 0', 'date', 'item_id', 'normal_price', 'price', 'quantity', 'store_id']
  ✓ actual_matrix        (3520, 4)       ['Un

## Sales LONG format 

In [9]:
# Check actual column names in each dataset
print("📋 ACTUAL COLUMN NAMES IN EACH DATASET:\n")
for name, df in dfs.items():
    print(f"{name.upper()}:")
    print(f"  Columns: {list(df.columns)}")
    print(f"  Shape: {df.shape}")
    print()

📋 ACTUAL COLUMN NAMES IN EACH DATASET:

SALES:
  Columns: ['Unnamed: 0', 'date', 'item_id', 'quantity', 'price_base', 'sum_total', 'store_id']
  Shape: (743268, 7)

PRICE_HISTORY:
  Columns: ['Unnamed: 0', 'date', 'item_id', 'price', 'code', 'store_id']
  Shape: (69863, 6)

DISCOUNTS_HISTORY:
  Columns: ['Unnamed: 0', 'date', 'item_id', 'sale_price_before_promo', 'sale_price_time_promo', 'promo_type_code', 'doc_id', 'number_disc_day', 'store_id']
  Shape: (374674, 9)

CATALOG:
  Columns: ['Unnamed: 0', 'item_id', 'dept_name', 'class_name', 'subclass_name', 'item_type', 'weight_volume', 'weight_netto', 'fatness']
  Shape: (21981, 9)

STORES:
  Columns: ['store_id', 'division', 'format', 'city', 'area']
  Shape: (4, 5)

ONLINE:
  Columns: ['Unnamed: 0', 'date', 'item_id', 'quantity', 'price_base', 'sum_total', 'store_id']
  Shape: (112341, 7)

MARKDOWNS:
  Columns: ['Unnamed: 0', 'date', 'item_id', 'normal_price', 'price', 'quantity', 'store_id']
  Shape: (898, 7)

ACTUAL_MATRIX:
  Colum

## Data Quality Checks

In [10]:
# Let's look at sample data from key files
print("\n📊 SAMPLE DATA:\n")
print("SALES (main transaction data):")
print(dfs['sales'].head(3))
print("\nSTORES (store info):")
print(dfs['stores'].head())


📊 SAMPLE DATA:

SALES (main transaction data):
   Unnamed: 0        date       item_id  quantity  price_base  sum_total  \
0    16277163  2023-02-22  ef09dbc9fa66       2.0       44.91      89.82   
1     1022837  2024-03-09  95416d766ab9       4.0       54.90     219.60   
2    21699470  2024-09-19  65093e8d67e6       1.0      401.00     401.00   

   store_id  
0         2  
1         1  
2         4  

STORES (store info):
   store_id division    format   city  area
0         4     Div1  MaxiEuro  City3  1887
1         1     Div1   Regular  City1  1200
2         2     Div2  MaxiEuro  City2  1500
3         3     Div2   Regular  City4  1000


## Merge Datasets

In [11]:
# ✓ Data is ALREADY in long format (not wide format)
# Each row = one transaction (date, item_id, store_id, quantity, price)
# So we DON'T need to melt/pivot

print("\n" + "="*70)
print("✓ DATA FORMAT CHECK")
print("="*70)
print(f"Data is already in LONG FORMAT ✓")
print(f"  No need for pivot/melt operations")
print(f"  Each row = 1 transaction on 1 date")
print(f"  We can directly merge on (item_id, store_id, date)")
print("="*70)


✓ DATA FORMAT CHECK
Data is already in LONG FORMAT ✓
  No need for pivot/melt operations
  Each row = 1 transaction on 1 date
  We can directly merge on (item_id, store_id, date)


## Save Final Dataset

In [12]:
# Use the complete pipeline to build final dataset
print("\n" + "="*70)
print("🚀 BUILDING MASTER DATASET FROM ALL 8 CSV FILES")
print("="*70 + "\n")

final_df = merge_datasets(dfs)
final_df = handle_missing_values(final_df)

# Save to processed folder
os.makedirs('../data/processed', exist_ok=True)
final_df.to_csv('../data/processed/final_dataset.csv', index=False)

print(f"\n✅ Master dataset built and saved!")
print(f"📊 Final dataset shape: {final_df.shape}")
print(f"📁 Saved to: ../data/processed/final_dataset.csv")
print("\n" + "="*70)


🚀 BUILDING MASTER DATASET FROM ALL 8 CSV FILES


🔄 Merging datasets...

  1. Start with SALES as main table
     Shape: (743268, 6)
  2. Merge with STORES (on store_id)
     Shape: (743268, 10)
  3. Merge with CATALOG (on item_id)
     Shape: (743268, 17)
  4. Merge with PRICE_HISTORY (on item_id, store_id, date)
     Shape: (743268, 19)
  5. Merge with DISCOUNTS (aggregate on item_id, store_id, date)
     Shape: (743268, 22)
  6. Merge with ONLINE SALES (aggregate on item_id, store_id, date)
     Shape: (743268, 25)
  7. Merge with MARKDOWNS (aggregate on item_id, store_id, date)
     Shape: (743268, 28)

✓ All merges complete!

📊 Handling missing values...

  Missing values before: 12899306
  Missing values after:  0

✅ Master dataset built and saved!
📊 Final dataset shape: (743268, 28)
📁 Saved to: ../data/processed/final_dataset.csv

